## 프롬프트 생성하는 방법
1. PromptTemplate.from_template() 매소드 사용해서 만들기
2. PromptTemplate 객체 생성과 동시에 프롬프트 만들기
    - input_variables : 명시적으로 {} 안의 변수명 지정
    - partial_variables : 함수를 부분적으로 사용(항상 공통된 방식으로 가져오고 싶은 변수)
3. ChatPromptTemplate : 대화목록을 프롬프트로 주입하고자 할 때 활용
    - 메시지는 튜플(tuple) 형식으로 구성하며, (role, message) 로 구성하여 리스트로 생성
    

In [ ]:
from langchain_ollama import ChatOllama
# LLM Model 
gemma = ChatOllama(model = "gemma3:latest", temperature=0.3 )
qwen = ChatOllama(model = "qwen3:1.7b", temperature=0.3 )

response1 = gemma.invoke("탄소의 원자번호는 뭐야?")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# LLM 지정 
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4.1",  # 모델명
)

# ChatPromptTemplate 이용해 대화목록 형식으로 프롬프트 주입 
chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        # conversation 이라는 변수명으로 대화목록 프롬프트 주입
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)
print(formatted_chat_prompt)
chain = chat_prompt | llm | StrOutputParser()
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)


System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


'인사, 입사, 자기소개, 반가움, 환영'

## json splitter

In [ ]:
from langchain_text_splitters import RecursiveJsonSplitter
import json
import requests

with open("../data/articles.json", 'r', encoding='utf-8') as file:
    json_data = json.load(file)[0]
# json_data = requests.get("https://api.smith.langchain.com/openapi.json").json()
# print(json_data)


# JSON 데이터를 최대 300 크기의 청크로 분할하는 RecursiveJsonSplitter 객체를 생성합니다.
splitter = RecursiveJsonSplitter(max_chunk_size=20)

# JSON 데이터를 재귀적으로 분할합니다. 작은 JSON 조각에 접근하거나 조작해야 하는 경우에 사용합니다.
json_chunks = splitter.split_json(json_data=json_data)

# JSON 데이터를 기반으로 문서를 생성합니다.
docs = splitter.create_documents(texts=[json_data])

# JSON 데이터를 기반으로 문자열 청크를 생성합니다.
texts = splitter.split_text(json_data=json_data)

# 첫 번째 문자열을 출력합니다.
print(docs[0].page_content)
print("===" * 20)
# 분할된 문자열 청크를 출력합니다.
print(texts[0])
print("===" * 20)

{"\uc2e0\ubb38\uc0ac": "\ub274\uc2dc\uc2a4", "\uae30\uc0ac\uc81c\ubaa9": "\uae40\uc7ac\uc12d \"\uae40\ubbfc\uc11d '\ub2e8\uc2dd\uc740 \uc544\ub2c8\uc8e0?' \ub098\uacbd\uc6d0 \uc870\ub871\u2026\ud55c \ubc29 \uba39\uc600\ub2e4\""}
{"\uc2e0\ubb38\uc0ac": "\ub274\uc2dc\uc2a4", "\uae30\uc0ac\uc81c\ubaa9": "\uae40\uc7ac\uc12d \"\uae40\ubbfc\uc11d '\ub2e8\uc2dd\uc740 \uc544\ub2c8\uc8e0?' \ub098\uacbd\uc6d0 \uc870\ub871\u2026\ud55c \ubc29 \uba39\uc600\ub2e4\""}


IndexError: list index out of range

In [ ]:
import json
def print_korean_json(data):
    if isinstance(data, dict):
        print(json.dumps(data, ensure_ascii=False, indent=2))
    elif isinstance(data, str):
        try:
            # 문자열이 실제 json이라면 파싱
            obj = json.loads(data)
            print(json.dumps(obj, ensure_ascii=False, indent=2))
        except Exception:
            # 그래도 안 되면 unicode_escape 시도
            try:
                print(data.encode('utf-8').decode('unicode_escape'))
            except Exception:
                print(data)
    else:
        print(data)
# 사용 예시
# print_korean_json(texts[0])
# print_korean_json(docs[0].page_content)

# Vector Store 
## Vector Store 사용 이b유
1. 빠른 검색 속도: 임베딩 벡터들을 효과적으로 저장하고 색인화함으로써, 대량의 데이터 중에서도 관련된 정보를 빠르게 검색할 수 있습니다.
2. 스케일러빌리티: 데이터가 지속적으로 증가함에 따라, 벡터스토어는 이를 수용할 수 있는 충분한 스케일러빌리티를 제공해야 합니다. 효율적인 저장 구조는 데이터베이스의 확장성을 보장하며, 시스템의 성능 저하 없이 대규모 데이터를 관리할 수 있도록 합니다.
3. 의미 검색(Semantic Search) 지원: 키워드 기반 검색이 아닌 사용자의 질문과 의미상으로 유사한 단락을 조회해야하는데, 벡터스토어는 이러한 기능을 지원합니다. 텍스트 자체가 저장되는 DB의 경우 키워드 기반 검색에 의존해야 하는 한계성이 있지만, 벡터스토어는 의미적으로 유사한 단락 검색을 가능케합니다.

In [29]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path='../data/articles.json',
    jq_schema='.[]',
    text_content=False)

data = loader.load()

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)
splite_data = loader.load_and_split(text_splitter)
print(len(splite_data))

7259


In [46]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma

# DB 생성
# db = Chroma.from_documents(
#     documents=splite_data, embedding=OpenAIEmbeddings(), collection_name="test_db"
# )

# 저장할 경로 지정
DB_PATH = "./test_db"

# 디스크에 저장 
persist_db = Chroma.from_documents(
    splite_data, OpenAIEmbeddings(chunk_size=500), persist_directory=DB_PATH, collection_name="test_db"
)

persist_db.get()

{'ids': ['5a405ab3-3fc1-4dfa-b356-d4b167649947',
  '583ac0dd-ca2b-4b05-8b1d-0459540c17b7',
  'd1e898bc-6a2a-4ae0-bf3c-48e8fd0d7ab8',
  'eaf91dbc-98fb-4755-a451-d7f9a8d8f075',
  '841c410c-339f-4ec8-9284-c8aa0a3a03c4',
  'ee275e7d-5162-4085-b6b5-059b871fe73d',
  '3b7f9774-a6ab-49a5-8878-537ce007be76',
  '25f25398-7b5b-4656-914b-41e6c16803fc',
  '5c480367-e547-43a9-af02-9c27063a1541',
  '83b6f29f-14aa-4d6d-b99e-17ea78cf54eb',
  '23ba3bfd-52bb-4ad9-9013-111bdbe33dab',
  'c8cdf229-e99d-4f3e-be2d-7d495bbc98a6',
  'c7336b06-12e3-48ef-bb95-b8458864bdd2',
  '2d79714c-5c68-41e5-92f9-26fb0dd38a31',
  'e269722a-98b0-4c3e-a77f-bfa84e788a18',
  'ac56ecdc-6506-4cf9-a767-2fae6539e8f6',
  '92c8e1f3-d838-465a-b38d-b738fa543d4f',
  '6c09713a-a874-4695-a086-0bf3cc25916f',
  '90656570-dda5-4aa5-93e3-d4516a372c00',
  '3144a8bc-42df-484f-8e22-d36f6402c6fe',
  'fee5428f-45db-4458-96bd-dd6edc5eb37c',
  '0cba5adb-77aa-482b-a69c-59c8e696f3f4',
  '9bf3c9bf-188c-4ac0-bfb5-55feed504783',
  '67af1cd7-7ee1-4981-a0d5-

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=OpenAIEmbeddings(chunk_size=200),
    collection_name="test_db",
)
retriever = persist_db.as_retriever()
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")  # OpenAI 언어 모델 초기화
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("기사 아무거나 소개해줘"))

주어진 문맥에서 소개할 수 있는 기사는 다음과 같습니다:

1. [기사 링크 1](https://n.news.naver.com/mnews/article/417/0001086696)
2. [기사 링크 2](https://n.news.naver.com/mnews/article/018/0006053775)

이 링크를 통해 기사를 확인해 보세요.


# Runnalbe 인터페이스
parser, llm, prompt 전부 Runnalbe 인터페이스를 구현한 컴포넌트다.
